## Loading the Data

In [1]:
from datasets import load_dataset
dataset = load_dataset('billingsmoore/tibetan-to-english-translation-dataset')

In [2]:
dataset = dataset['train'].train_test_split(.15)

In [3]:
dataset['train'][0]

{'tibetan': 'སྒྲིབ་དག་བྱིན་རླབས་དབང་བསྐུར་ཐོབ།།',
 'phonetic': 'drib dak jinlab wangkur tob',
 'english': 'Purify obscurations confer blessings and empowerment'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [4]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "billingsmoore/tibetan-to-english-translation"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Preprocess Data

The dataset can now be tokenized for training.

In [5]:
def translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['tibetan']]
    translation_targets = [example for example in examples['english']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return translation_model_inputs


In [6]:
def transliteration_preprocess_function(examples):
    # Prepare transliteration inputs and targets
    transliteration_inputs = ['Transliterate: ' + example for example in examples['tibetan']]
    transliteration_targets = [example for example in examples['phonetic']]
    
    # Tokenize transliteration inputs and targets
    transliteration_model_inputs = tokenizer(transliteration_inputs, text_target=transliteration_targets, 
                                             max_length=300, truncation=True, padding="max_length")
    
    return transliteration_model_inputs

In [7]:
translation_tokenized_dataset = dataset.map(translation_preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map:   0%|          | 0/12184 [00:00<?, ? examples/s]

In [8]:
transliteration_tokenized_dataset = dataset.map(transliteration_preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map:   0%|          | 0/12184 [00:00<?, ? examples/s]

In [9]:
from datasets import concatenate_datasets

tokenized_dataset = {}

tokenized_dataset['train'] = concatenate_datasets([translation_tokenized_dataset['train'], transliteration_tokenized_dataset['train']])

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [12]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


True

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"dual-task-add-both",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=1
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset={'translation':translation_tokenized_dataset['test'], 'transliteration':transliteration_tokenized_dataset['test']},
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/17261 [00:00<?, ?it/s]

  0%|          | 0/34521 [00:00<?, ?it/s]

  0%|          | 0/69042 [00:00<?, ?it/s]

{'loss': 0.1273, 'grad_norm': 0.19353359937667847, 'learning_rate': 0.0002978274094029721, 'epoch': 0.01}
{'loss': 0.0756, 'grad_norm': 0.5356396436691284, 'learning_rate': 0.0002956548188059442, 'epoch': 0.01}
{'loss': 0.0577, 'grad_norm': 0.29399168491363525, 'learning_rate': 0.00029348222820891625, 'epoch': 0.02}
{'loss': 0.0457, 'grad_norm': 0.4854101836681366, 'learning_rate': 0.0002913096376118884, 'epoch': 0.03}
{'loss': 0.0448, 'grad_norm': 0.27622082829475403, 'learning_rate': 0.0002891370470148605, 'epoch': 0.04}
{'loss': 0.0373, 'grad_norm': 0.509633481502533, 'learning_rate': 0.0002869644564178326, 'epoch': 0.04}
{'loss': 0.0443, 'grad_norm': 0.3819696605205536, 'learning_rate': 0.0002847918658208047, 'epoch': 0.05}
{'loss': 0.0354, 'grad_norm': 0.24780891835689545, 'learning_rate': 0.00028261927522377683, 'epoch': 0.06}
{'loss': 0.0349, 'grad_norm': 0.7833503484725952, 'learning_rate': 0.0002804466846267489, 'epoch': 0.07}
{'loss': 0.0326, 'grad_norm': 0.1322004646062851, 

...

  0%|          | 0/1523 [00:00<?, ?it/s]

{'eval_translation_loss': 0.00932574737817049, 'eval_translation_bleu': 65.4073, 'eval_translation_gen_len': 14.3828, 'eval_translation_runtime': 1841.5002, 'eval_translation_samples_per_second': 6.616, 'eval_translation_steps_per_second': 0.827, 'epoch': 1.0}


  0%|          | 0/1523 [00:00<?, ?it/s]

{'eval_transliteration_loss': 0.008420788682997227, 'eval_transliteration_bleu': 66.0247, 'eval_transliteration_gen_len': 17.6756, 'eval_transliteration_runtime': 1863.538, 'eval_transliteration_samples_per_second': 6.538, 'eval_transliteration_steps_per_second': 0.817, 'epoch': 1.0}


KeyError: "The `metric_for_best_model` training argument is set to 'eval_loss', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_translation_loss', 'eval_translation_bleu', 'eval_translation_gen_len', 'eval_translation_runtime', 'eval_translation_samples_per_second', 'eval_translation_steps_per_second', 'epoch', 'eval_transliteration_loss', 'eval_transliteration_bleu', 'eval_transliteration_gen_len', 'eval_transliteration_runtime', 'eval_transliteration_samples_per_second', 'eval_transliteration_steps_per_second']. Consider changing the `metric_for_best_model` via the TrainingArguments."